# GraphQL Endpoints

In [4]:
base_url = 'https://base.easscan.org/graphql'
base_sepolia_url = 'https://base-sepolia.easscan.org/graphql' # testnet

schemaID = '0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68' # OLI v1.0.0 schema

### Retrieve The Latest Attestations For The OLI Label Pool

In [5]:
import requests

def query_attestations(endpoint: str, schemaId: str, count: int = 10) -> dict:
    query = """
    query Attestations($where: AttestationWhereInput, $take: Int) {
      attestations(take: $take, where: $where) {
        attester
        data
        decodedDataJson
        expirationTime
        id
        ipfsHash
        isOffchain
        recipient
        refUID
        revocable
        revocationTime
        revoked
        schemaId
        time
        timeCreated
        txid
      }
    }
    """
    
    variables = {
        "where": {
            "schemaId": {
                "equals": schemaId
            }
        },
        "take": count
    }
    
    headers = {
        "Content-Type": "application/json"
    }
    
    response = requests.post(endpoint, json={"query": query, "variables": variables}, headers=headers)
    
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"GraphQL query failed with status code {response.status_code}: {response.text}")

# Example usage:
result = query_attestations(base_url, schemaID)
print(result)


{'data': {'attestations': [{'attester': '0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56', 'data': '{"sig":{"version":2,"uid":"0x0001326295e5b77c04490636fa2bdcbbd7c8567f1684884c5f273596d022c133","domain":{"name":"EAS Attestation","version":"1.0.1","chainId":"8453","verifyingContract":"0x4200000000000000000000000000000000000021"},"primaryType":"Attest","message":{"version":2,"recipient":"0x29faf5905bff9cfcc7cf56a5ed91e0f091f8664b","expirationTime":"0","time":"1739304486","revocable":true,"schema":"0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68","refUID":"0x0000000000000000000000000000000000000000000000000000000000000000","data":"0x0000000000000000000000000000000000000000000000000000000000000040000000000000000000000000000000000000000000000000000000000000008000000000000000000000000000000000000000000000000000000000000000096569703135353a31300000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000003f7b22636f6e74726163745f6e616

In [8]:
import json
import pandas as pd

# create a DataFrame from the result and extract the decoded data into separate columns
df = pd.DataFrame(result['data']['attestations'])
def extract_decoded_data(json_str):
    data = json.loads(json_str)
    return {item["name"]: json.loads(item["value"]["value"]) if item["name"] == "tags_json" else item["value"]["value"] for item in data}
df = df.join(df["decodedDataJson"].apply(lambda x: pd.Series(extract_decoded_data(x))))
df

,attester,data,decodedDataJson,expirationTime,id,ipfsHash,isOffchain,recipient,refUID,revocable,revocationTime,revoked,schemaId,time,timeCreated,txid,chain_id,tags_json
0,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x0001326295e5b77c0...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x0001326295e5b77c04490636fa2bdcbbd7c8567f1684...,QmTm8Qiv6NuXm8EbCB9rPTg6jqyTXgSfrV2YHwJQHLb99v,True,0x29FAF5905bfF9Cfcc7CF56a5ed91E0f091F8664B,0x00000000000000000000000000000000000000000000...,True,0,False,0xb763e62d940bed6f527dd82418e146a904e62a297b8f...,1739304486,1739304487,,eip155:10,"{'contract_name': 'bankless', 'usage_category'..."
1,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x00019aeadab0e911a...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x00019aeadab0e911ac52fc2d765aeee01c2825928c73...,QmV6wbw8osKsnays7e2u3rXkykYKaGV5gzkDbCv9AEgZJz,True,0x6C9939519D00096b0Bc2c26B13C408a994D8f910,0x00000000000000000000000000000000000000000000...,True,0,False,0xb763e62d940bed6f527dd82418e146a904e62a297b8f...,1739317991,1739317991,,eip155:42161,{'usage_category': 'fungible_tokens'}
2,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x0002256bb4a9a6790...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x0002256bb4a9a679061be2bcce049b25d9f6765885c5...,QmNh6aPzQqJjv3sgUXkbrPQjaAhXWPvySPZjeDTN3KMf4x,True,0xFB461E57AbCF2AE710d24E38F9f2d76dcb0671dc,0x00000000000000000000000000000000000000000000...,True,0,False,0xb763e62d940bed6f527dd82418e146a904e62a297b8f...,1739328781,1739328784,,eip155:42161,{'usage_category': 'non_fungible_tokens'}
3,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x00028e6ecb6fb520c...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x00028e6ecb6fb520ca57b50a2b6c22755e95b4ded2ee...,QmSohMw1i5zbLDfQZd8NNLT1dV9FXkdD9tEzX1o2tCJyPy,True,0xC9Da4521b580C8603056921C21dcb029fA36C4f2,0x00000000000000000000000000000000000000000000...,True,0,False,0xb763e62d940bed6f527dd82418e146a904e62a297b8f...,1739363930,1739363931,,eip155:8453,"{'is_eoa': False, 'is_contract': True, 'contra..."
4,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x000357a8210ae2b28...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x000357a8210ae2b2889750d4bb283660e8e68c7329bd...,QmVrUQGHMLjGG9gMXHVoqe7xzNNRRGfqBYrzdy5KpULKqp,True,0xccC4B1d35Ce9a819737d0C821c6BF70f9fE7097F,0x00000000000000000000000000000000000000000000...,True,0,False,0xb763e62d940bed6f527dd82418e146a904e62a297b8f...,1739364521,1739364521,,eip155:10,{'usage_category': 'non_fungible_tokens'}
5,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x0003b8d89713065f7...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x0003b8d89713065f7a45608515cbd78711a90ede9551...,QmdNUVwaXyZheT3Fyrw4Jn4H6CetQysb443UFdcFT8hQTK,True,0xB59De922FB30Acd1000F0Df0baFf916193127bb5,0x00000000000000000000000000000000000000000000...,True,0,False,0xb763e62d940bed6f527dd82418e146a904e62a297b8f...,1739359931,1739359932,,eip155:42161,"{'contract_name': 'sushi', 'usage_category': '..."
6,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x00068a35ab24e635f...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x00068a35ab24e635f6ac0233da7403e86f2c520173fc...,QmSQ2YWTNd69fxm5se8kD2F52jn7Dazm64s4DuHLK73s3T,True,0x5455D1BB398C3E2C97C28393c34E9733fB1F1336,0x00000000000000000000000000000000000000000000...,True,0,False,0xb763e62d940bed6f527dd82418e146a904e62a297b8f...,1739313271,1739313272,,eip155:42161,"{'contract_name': 'sushi', 'usage_category': '..."
7,0xA725646c05e6Bb813d98C5aBB4E72DF4bcF00B56,"{""sig"":{""version"":2,""uid"":""0x0006c661f7b131c3c...","[{""name"":""chain_id"",""type"":""string"",""signature...",0,0x0006c661f7b131c3c76ba22252e3b1495cd251dd88d9...,Qmd4CrMkAK851CKEkSGKA46fCKDfcGrNJggCD8iqygckcc,True,0x1C0E81013E